# Numba

Para entender que es numba hay que definir el siguiente concepto:

- Just in time compiler: *The Just-In-Time (JIT) compiler is a component of the runtime environment that improves the performance of applications by compiling bytecodes to native machine code at run time*
    - Se ejecuta en tiempo de ejecucion
    - transforma los codigos de byte en codigo nativo de maquina

Para entender un poco mas este concepto tenemos esta discusion en <a href='https://stackoverflow.com/questions/95635/what-does-a-just-in-time-jit-compiler-do'>Stack Overflow </a>

- Un compilador normal, compila todo el programa antes de la primera corrida.
- Un compilador justo a tiempo corre el programa justo en el momento de la ejecucion del mismo.
- Un compilador normal por otro lado tendra que compilara el codigo en cada llamada
- El compilador justo a tiempo solo compilara nuevas piezas de codigo introducidas en cada llamada, mientras que las preexistentes no seran compiladas nuevamente.

## Ahora si, que es numba?
<a href='https://numba.pydata.org/numba-doc/latest/user/5minguide.html'>Documentacion Oficial</a>
Numba es un compilador justo a tiempo que funciona mejor para codigo que utilize arreglos y funciones de numpy.

Numba funcina con decoradores que se deben utilizar sobre las funciones para instruirle a Numba que las compile.

Cuando se hace una llamada a una funcion decorada con Numba, esta se compila a codigo maquina 'justo a tiempo' para su ejecucion. Este codigo ya esta en codigo maquina para una ejecucion veloz.

### Instalacion:
pip install numba

### Cuando utilizar Numba?
La documentacion oficial indica que: *This depends on what your code looks like, if your code is numerically orientated (does a lot of math), uses NumPy a lot and/or has a lot of loops, then Numba is often a good choice*

Numba es una buena opcion si:

- Nuestro codigo utiliza muchas funciones de numpy
- Tiene orientacion numerica
- Tiene muchos loops
- En contraposicion no nos va a servir en absoluto para codigo de pandas por ejemplo.
- Es clave que entregemos el mismo tipo de argumentos para nuestras funciones. Si nuestras funciones recibian enteros pues habra una rapida ejecucion si se otorgan enteros (distintos) a las entradas.

### Como se utiliza numba?

1. Importamos su decorador: from numba import jit
2. Utilizamos su decorador en alguna funcion @jit
3. Opciones del decorador:
    - nopython = True --> Hace que se compile la funcion decorada para no involucrar al interprete de python (esta es la funcion recomendada)
    - forceobj = True ---> Compilacion por defecto si no utilizamos ninguna opcion para el decorador (no recomendado)
    - Se usan asi: @jit(nopython=True) // @jit
    - @jit(forceobj = True)

Cual es la diferencia entre nopython y forceobj? Primero hay que tener en cuenta lo que significa el interprete de un lenguaje:

*An interpreter is a program that directly executes the instructions in a high-level language, without converting it into machine code. In programming, we can execute a program in two ways. Firstly, through compilation and secondly, through an interpreter. The common way is to use a compiler.*

Si vos utilizas nopython = True no se utiliza el interprete de python, si no que compila directamente y lo hace infiriendo los tipos de los inputs y outputs, si se utilizan objetos mas complejos, con tipos no conocidos por Numba, entonces falla y da un error.

Si vos **NO** utilizas nopython = True , es decir, solo el decorador, vamos a tener que la compilacion se hace con un warning, pero se hace

Si vos utilizas forceobj = True Entonces ya estarias utilizando el interprete, aqui todos los objetos de python son admitidos. Sin embargo debido al uso del interprete puede, y repito puede, llegar a tardar mas que la opcion de nopython.

Por default : @jit = @jit() Es decir estas dos opciones son lo mismo. 

<a href='https://numba.pydata.org/numba-doc/0.17.0/glossary.html#term-object-mode'>Doc oficial</a>

<a href='https://numba.pydata.org/numba-doc/0.17.0/reference/compilation.html'>Doc oficial 2</a>

Mas a fondo: <a href='https://stackoverflow.com/questions/71510827/numba-when-to-use-nopython-true'>Stack Overflow</a>

### Como podemos medir la performance de Numba?

Primero hay que tener en cuenta que Numba **toma tiempo** en compilar la funcion para los tipos de argumentos que utiliza antes de ejecutar el codigo maquina.

Al volverla a llamar a la funcion Numba cachea el codigo de la funcion y la utiliza para el tipo de los argumentos del mismo tipo entregados, se utiliza la version cacheada para ejecutar el codigo sin necesidad de recompilar el codigo.

Podemos incluir un timer para ver cuanto tarda en ejecutarse el codigo. Para ello podemos utilizar el modulo time de python.

### Como funciona numba?
*Numba reads the Python bytecode for a decorated function and combines this with information about the types of the input arguments to the function. It analyzes and optimizes your code, and finally uses the LLVM compiler library to generate a machine code version of your function, tailored to your CPU capabilities. This compiled version is then used every time your function is called.*

- La libreria de LLVM es el backend de Numba

In [2]:
from numba import jit
import numpy as np 
import time

x = np.arange(100).reshape(10, 10)

@jit(nopython=True)
def go_fast(a): # Function is compiled and runs in machine code
    trace = 0.0
    for i in range(a.shape[0]):
        trace += np.tanh(a[i, i])
    return a + trace

# DO NOT REPORT THIS... COMPILATION TIME IS INCLUDED IN THE EXECUTION TIME!
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (with compilation) = %s" % (end - start))

# NOW THE FUNCTION IS COMPILED, RE-TIME IT EXECUTING FROM CACHE
start = time.time()
go_fast(x)
end = time.time()
print("Elapsed (after compilation) = %s" % (end - start))

# OTRA MANERA DE TOMAR EL TIEMPO:

%timeit go_fast(x)

Elapsed (with compilation) = 0.2397623062133789
Elapsed (after compilation) = 0.0
1.43 µs ± 63.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Ahora que sabemos como funciona Numba podemos entender un poco mejor este ejemplo. En este ejemplo se elabora una funcion f(x) que suma los elementos dentro de la lista x.

Tenemos dos funciones g21, g22 y g23 una esta decorada y la otra no.

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
import numba
@numba.jit(nopython=True, cache=True)
def f(x):
    summ = 0
    for i in x:
        summ += i
    return summ

@numba.jit(nopython=True)
def g21(N, locs):
    rvs = np.random.normal(loc=locs, scale=locs, size=N)
    res = f(rvs)
    return res

@numba.jit
def g22(N, locs):
    rvs = np.random.normal(loc=locs, scale=locs, size=N)
    res = f(rvs)
    return res

@numba.jit(forceobj=True)
def g23(N, locs):
    rvs = np.random.normal(loc=locs, scale=locs, size=N)
    res = f(rvs)
    return res

N = 10_000

# g21(N, np.linspace(0,1,N))  # Da Error
# g21(N, 3)
# g22(N, 3)
g22(N, np.linspace(0,1,N)) #Compila con Warning
# g23(N, 3)
# g23(N, np.linspace(0,1,N)) #Compila sin warning



C:\Users\Fede\AppData\Local\Temp\ipykernel_6940\3171942240.py:18: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit
C:\Users\Fede\AppData\Local\Temp\ipykernel_6940\3171942240.py:18: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "g22" failed type inference due to: No implementation of function Function(<built-in method normal of numpy.random.mtrand.RandomState object at 0x0000028D1E8BEB40>) found for signature:
 
 >>> normal(loc=array(float64, 1d, C), scale=array(float64, 1d, C), size=int64)
 
There are 8 candidate implementations:
      - Of which 2 did not match due to:
      Overload in function 'np_gauss_impl0'

5036.018016375639

## Inferencia de tipos en Numba

Por defecto, numba infiere los tipos de las funciones sobre las que se aplica el decorador. Entonces, si volvemos a correr la funcion con argumentos del mismo tipo numba tardara menos. En cambio si le otorgamos inputs de otro tipo, numba volvera a tardar en su primera corrida con inputs de ese tipo, pero tardara menos en las subsecuentes.

Numba ira guardando en la cache todos los tipos de los inputs con los que hayamos ejecutado la misma.

In [18]:
@jit(nopython=True)  # numba.njit
def add(a, b):
    return a + b

#La primera corrida se hace con enteros
start = time.time()
add(1,1)
end = time.time()
print("Elapsed  = %.7f" % (end - start))
#La segunda corrida ya tardamos menos 
start = time.time()
add(5,7)
end = time.time()
print("Elapsed  = %.7f" % (end - start))

#Si ahora llamo con float tendre un tiempo largo 
start = time.time()
add(5.,7)
end = time.time()
print("Elapsed  = %.7f" % (end - start))
#En una subsecuente llamada del mismo tipo se reduce el tiempo
start = time.time()
add(9.,9)
end = time.time()
print("Elapsed  = %.7f" % (end - start))

#Ojo que el primero era el flotante, si vos invertis, primero int y luego float ya tenes otra combinacion y alli tardara
start = time.time()
add(9,9.)
end = time.time()
print("Elapsed  = %.7f" % (end - start))

#Podemos acceder a las signatures utilizadas con el siguiente comando
print(add.nopython_signatures)

Elapsed  = 0.0869033
Elapsed  = 0.0000000
Elapsed  = 0.1216795
Elapsed  = 0.0000000
Elapsed  = 0.0617104
[(int64, int64) -> int64, (float64, int64) -> float64, (int64, float64) -> float64]


## Numba types and signatures
Si tenes una funcion decorada con numba podes indicar el tipo de los inputs. Despues numba puede inferir el tipo del output o podes tambien indicar el tipo del output tal y como se indica aca: <a href='https://numba.pydata.org/numba-doc/latest/reference/jit-compilation.html'>Numba Functions</a>

Basicamente hay tres formas de declarar estos tipos (primero los vas a tener que importar desde numba, ej: from numba import int32, o bien utilizrlos como numba.int32)

1. Utilizarlos como tupla: @jit(["int32(int32)", "float32(float32)"], nopython=True) <---- Los tipos presentes en numba los podes encontrar en <a href='Tipos en Numba'>Tipos de Numba</a> . 

## Funciones universales en Numba

En este contexto estamos hablando de funciones universales de Numpy. Puede verse la <a href='https://numpy.org/doc/stable/reference/ufuncs.html'>Documentacion oficial</a>

*A universal function (or ufunc for short) is a function that operates on ndarrays in an element-by-element fashion, supporting array broadcasting, type casting, and several other standard features.*

Hay dos maneras de implementar estas funciones en Numba deacuerdo a <a href='https://numba.pydata.org/numba-doc/latest/user/vectorize.html#creating-numpy-universal-functions'>Documentacion oficial</a> de Numba.

- Decorador @vectorize : Para funciones universales que operan en escalares
- Decorador @guvectorize : Para funciones que operan en mayor dimension y escalares